In [1]:
#cifar 10 data set
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten,BatchNormalization,GaussianNoise

from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers

import keras.backend as K
from keras.optimizers import rmsprop

from keras.utils import np_utils
from keras.datasets import cifar10
import numpy as np

import matplotlib.pyplot as plt

from keras import backend as K
K.set_image_dim_ordering('tf')




Using TensorFlow backend.


In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [3]:
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
std

64.150024

In [0]:
nClasses = 10
y_train = np_utils.to_categorical(y_train,nClasses)
y_test = np_utils.to_categorical(y_test,nClasses)

In [5]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [0]:
input_shape = (32,32,3)
weight_decay=1e-3

In [0]:
def createModel():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3),kernel_regularizer=regularizers.l2(weight_decay), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(GaussianNoise(0.2))    
        
    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3),kernel_regularizer=regularizers.l2(weight_decay), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))  

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3),kernel_regularizer=regularizers.l2(weight_decay), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.7))
   
 
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(nClasses, activation='softmax'))
     
    return model

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)
batchsize = 256
dgen=datagen.flow(x_train, y_train, batch_size=batchsize)


In [9]:
K.clear_session()
model = createModel()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(opt_rms, loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
epoch = 20
model.fit_generator(dgen,steps_per_epoch=781,epochs=3*epoch,
                    verbose=0,validation_data=(x_test,y_test))

opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(opt_rms, loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit_generator(dgen,steps_per_epoch=781,epochs=epoch,
                    verbose=0,validation_data=(x_test,y_test))

opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(opt_rms, loss='categorical_crossentropy', 
              metrics=['accuracy'])

history=model.fit_generator(dgen,steps_per_epoch=781,epochs=50,
                    verbose=1,validation_data=(x_test,y_test))






Instructions for updating:
Use tf.cast instead.
Epoch 1/50
781/781 [==============================] - 138s 177ms/step - loss: 0.5163 - acc: 0.8729 - val_loss: 0.5335 - val_acc: 0.8670
Epoch 2/50
781/781 [==============================] - 137s 175ms/step - loss: 0.4985 - acc: 0.8770 - val_loss: 0.5247 - val_acc: 0.8712
Epoch 3/50
781/781 [==============================] - 136s 174ms/step - loss: 0.4918 - acc: 0.8780 - val_loss: 0.5337 - val_acc: 0.8685
Epoch 4/50
781/781 [==============================] - 137s 175ms/step - loss: 0.4834 - acc: 0.8781 - val_loss: 0.4991 - val_acc: 0.8757
Epoch 5/50
781/781 [==============================] - 138s 176ms/step - loss: 0.4801 - acc: 0.8788 - val_loss: 0.5090 - val_acc: 0.8755
Epoch 6/50
781/781 [==============================] - 136s 174ms/step - loss: 0.4782 - acc: 0.8795 - val_loss: 0.4921 - val_acc: 0.8768
Epoch 7/50
781/781 [==============================] - 137s 176ms/step - loss: 0.4773 - acc: 0.8786 - val_loss: 0.5778 - val_acc: 0.8500


In [0]:
# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'black',linewidth=3.0)
plt.plot(history.history['val_loss'],'black',ls = '--', linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
#plt.title('Loss Curves',fontsize=16)

In [0]:
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'black',linewidth=3.0)
plt.plot(history.history['val_acc'],'black',ls = '--',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18, 
           loc = 'lower right')
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
#plt.title('Accuracy Curves',fontsize=16)